# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as they are best equipped for this task.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "The task involves using statistical modeling and machine learning to identify trends and predict market movements for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL/"} 

Apple Inc. (AAPL) Stock Pric

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Tariff Updates Premium News Earnings Tech Housing Crypto Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Tariff Updates Premium News Earnings Tech Housing Crypto Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

Final Answer: 
After analyzing the historical prices of AAPL stock for the past month, we can see that the stock has experienced fluctuations in its price. The stock opened at $231.20 on February 12th and closed at $236.87 with a volume of 45,201,200. Throughout the month, the stock has seen highs and lows, with the highest closing price of $236.87 on February 12th and the lowest closing price of $227.65 on February 10th. 

It is important to note that the stock also paid a dividend of $0.25 on February 10th. Overall, the stock has shown some volatility in the past month, but it has managed to maintain an upward trend with some fluctuations.

> Finished chain.
 

After analyzing the historical prices of AAPL stock for the past month, we can see that the stock has experienced fluctuations in its price. The stock opened at $231.20 on February 12th and closed at $236.87 with a volume of 45,201,200. Throughout the month, the stock has seen highs and lows, with the highest closing price of 

I need to gather insights from the team to develop detailed execution plans for the approved trading strategies for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Analyze approved trading strategies for AAPL", "context": "I need a detailed analysis of the approved trading strategies for AAPL, taking into account current market conditions and optimal pricing based on the user's risk tolerance and trading preferences."}

> Entering new CrewAgentExecutor chain...
I need to gather detailed information on approved trading strategies for AAPL, taking into account current market conditions and optimal pricing based on risk tolerance and trading preferences.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/trading/top-strategies-for-trading-apple-aapl/"} 

Page Not Found
Skip to content
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ETFs
 Options and Derivatives
 Commodities
 Trading
 Automated Investing
 Broke

Final Answer: 

Detailed execution plans for approved trading strategies for AAPL:

1. Intraday Price Movements Strategy:
- For users with a higher risk tolerance and preference for day trading
- Monitor intraday price movements closely and execute trades based on momentum trading
- Utilize stop-loss orders to manage risk and capital preservation
- Target short-term gains based on intraday price fluctuations

2. Dollar-Cost Averaging Strategy:
- For users with a lower risk tolerance and preference for long-term investments
- Invest a fixed amount of money in AAPL stock at regular intervals
- Benefit from market volatility by averaging out the purchase price over time
- Hold onto the stock for the long term to capitalize on potential growth

3. Buy and Hold Strategy:
- For users with a conservative risk tolerance and long-term investment goals
- Purchase AAPL stock at the current market price and hold onto it for an extended period
- Monitor fundamental factors such as Apple's financial

Final Answer: 

Risk Analysis Report for AAPL Trading Strategies:

Intraday Price Movements Strategy:
- Potential Risks:
  - High volatility and sudden price movements
  - Increased transaction costs
- Mitigation Strategies:
  - Set strict stop-loss orders
  - Use technical analysis tools
  - Stay informed about market news

Dollar-Cost Averaging Strategy:
- Potential Risks:
  - Market downturns affecting average cost per share
  - Potential losses during bear markets
- Mitigation Strategies:
  - Diversify portfolio
  - Have a long-term investment horizon
  - Regularly monitor investments

Buy and Hold Strategy:
- Potential Risks:
  - Market volatility and company-specific risks
  - Potential loss of capital if stock underperforms
- Mitigation Strategies:
  - Conduct thorough research on AAPL
  - Diversify portfolio
  - Have a long-term investment outlook

Overall, traders and investors should carefully consider the risks associated with each trading strategy and implement appropriate 

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Intraday Price Movements Strategy:
- Potential Risks:
  - High volatility and sudden price movements
  - Increased transaction costs
- Mitigation Strategies:
  - Set strict stop-loss orders
  - Use technical analysis tools
  - Stay informed about market news

Dollar-Cost Averaging Strategy:
- Potential Risks:
  - Market downturns affecting average cost per share
  - Potential losses during bear markets
- Mitigation Strategies:
  - Diversify portfolio
  - Have a long-term investment horizon
  - Regularly monitor investments

Buy and Hold Strategy:
- Potential Risks:
  - Market volatility and company-specific risks
  - Potential loss of capital if stock underperforms
- Mitigation Strategies:
  - Conduct thorough research on AAPL
  - Diversify portfolio
  - Have a long-term investment outlook

Overall, traders and investors should carefully consider the risks associated with each trading strategy and implement appropriate risk management measures to protect their capital and achieve their financial goals. By understanding the potential risks and implementing the suggested mitigation strategies, traders can optimize their investment in AAPL stock.